
# Bu soruya cevap arıyoruz: **Daha iyi YOLO backbone nasıl yazılır?**
(Sistematik tasarım + kontrollü deney)

Bu notebook’un hedefi “random modül eklemek” değil.  
Amaç: YOLO backbone tasarımını **mühendislik kısıtları + veri istatistiği + controlled ablation** ile sistematik hale getirmek.

Bu soruya cevap arıyoruz:

- YOLO backbone’u hangi prensiplere göre tasarlarız?
- Stride / stage (C2–C5) / kanal artışı / blok tipi seçimi nasıl yapılır?
- Attention / norm / regularization nerede mantıklı?
- “Daha iyi”yi nasıl ölçeriz? (mAP_s/m/l + latency + VRAM)
- Deney düzeni nasıl kurulur? (tek değişken - tek deney)



---

## 0) Kısa özet: İyi backbone = doğru tradeoff

Backbone tasarımı şu optimizasyon problemidir:

\[
\max \; \text{mAP}_{s} + \text{mAP}_{m} + \text{mAP}_{l}
\quad \text{s.t.} \quad
\text{latency} \le X,\; \text{VRAM} \le Y
\]

Yani amaç sadece accuracy değil:
- **mAP_small/medium/large**
- **FPS / latency**
- **VRAM**
- (istenirse) parametre/FLOPs

Bir backbone’u “daha iyi” yapan şey:
- veri ve görev ihtiyacına göre doğru **stride planı**
- doğru **feature kalitesi**
- doğru **neck uyumu**
- doğru **maliyet**



---

## 1) YOLO mimarisini haritalandır: Backbone / Neck / Head

- Backbone: C2–C5 gibi farklı stride seviyelerinde feature üretir
- Neck: bu feature’ları birleştirir (FPN/PAN/PAFPN)
- Head: tahmin yapar (bbox/cls/obj)

Backbone iyileştirmenin “doğru” olması için:
1) Neck’in beklediği stride/kanal düzeniyle uyumlu olmalı  
2) mAP_small/medium/large hedeflerine göre ölçek piramidi doğru kurulmalı



---

## 2) Backbone tasarımının 4 ana düğümü

### 2.1) Stride planı (downsample schedule)
Kritik soru:
- Küçük nesne yoğun mu?  
  → erken downsample agresif olmasın, **P3 (stride 8)** kalitesi kritik

### 2.2) Kanal artış stratejisi
Kritik soru:
- kapasiteyi derinlik mi genişlik mi taşıyacak?

### 2.3) Blok tipi
Kritik soru:
- residual/csp/grouped/dw-pw gibi yapılar compute/accuracy dengesini belirler

### 2.4) Stabilizasyon / regularization
Kritik soru:
- batch küçük mü? BN mi GN mi?
- overfit var mı? DropPath/DropBlock?



---

## 3) “Veriye göre backbone” sezgisi: ilk bakılacak 5 istatistik

Backbone tasarımını dataset belirler. Eğitime başlamadan önce şunları çıkar:

1) Nesne boyutu dağılımı (small/medium/large)
2) Görüntü çözünürlüğü dağılımı
3) Sınıf dengesizliği
4) Overlap/crowd seviyesi
5) Blur/noise/low-light (domain zorluğu)

Bu istatistikler şunu söyler:
- P3/P4/P5 ağırlığı nasıl olmalı?
- ekstra P2 (stride 4) gerekir mi?
- neck fusion’u güçlendirmek mi gerekir?


In [1]:

# Template: veri istatistiğini çıkarmak için metrik listesi (dataset okumadan kontrol listesi)

dataset_stats_template = [
    "bbox area histogram (relative area)",
    "small/medium/large ratio",
    "bbox aspect ratio histogram",
    "image resolution histogram",
    "avg brightness/contrast distribution",
    "blur score distribution (var(Laplacian))",
    "class frequency (imbalance)",
    "overlap/crowd: mean IoU between boxes",
]
for s in dataset_stats_template:
    print("-", s)


- bbox area histogram (relative area)
- small/medium/large ratio
- bbox aspect ratio histogram
- image resolution histogram
- avg brightness/contrast distribution
- blur score distribution (var(Laplacian))
- class frequency (imbalance)
- overlap/crowd: mean IoU between boxes



---

## 4) Stride planı: backbone kalitesinin büyük kısmı burada

Tipik YOLO/Det backbone seviyeleri:

- C3: stride 8  (küçük)
- C4: stride 16 (orta)
- C5: stride 32 (büyük)

Eğer small-object baskınsa:
- P3/C3 tarafını güçlendirmek genelde daha verimli
- ama bunu “çözünürlüğü sonsuza kadar yüksek tutarak” değil:
  - daha iyi neck fusion
  - doğru augment
  - doğru label assignment/loss
  - sınırlı attention
ile yap

Aşağıdaki helper, nesne boyutuna göre hangi stride seviyesinin kritik olacağını kaba söyler.


In [2]:

def stride_recommend(obj_px, strides=(8,16,32), min_cells=2):
    # obj_px: nesnenin min(w,h) pikseli gibi kaba bir boyut
    # min_cells: nesne kaç feature cell kaplasın hedefi
    rec = [(s, obj_px/s) for s in strides]
    feasible = [r for r in rec if r[1] >= min_cells]
    if feasible:
        best = max(feasible, key=lambda x: x[0])
        return rec, f"Öneri: stride {best[0]} seviyesi yeterli (≈{best[1]:.2f} cell)."
    return rec, "Uyarı: Nesne çok küçük; stride 8 bile hedef cell'i tutmuyor. Çözünürlük/augment/neck iyileştirme düşün."

for obj in [6, 12, 20, 40, 80]:
    rec, msg = stride_recommend(obj)
    print(f"obj≈{obj}px -> {rec} | {msg}")


obj≈6px -> [(8, 0.75), (16, 0.375), (32, 0.1875)] | Uyarı: Nesne çok küçük; stride 8 bile hedef cell'i tutmuyor. Çözünürlük/augment/neck iyileştirme düşün.
obj≈12px -> [(8, 1.5), (16, 0.75), (32, 0.375)] | Uyarı: Nesne çok küçük; stride 8 bile hedef cell'i tutmuyor. Çözünürlük/augment/neck iyileştirme düşün.
obj≈20px -> [(8, 2.5), (16, 1.25), (32, 0.625)] | Öneri: stride 8 seviyesi yeterli (≈2.50 cell).
obj≈40px -> [(8, 5.0), (16, 2.5), (32, 1.25)] | Öneri: stride 16 seviyesi yeterli (≈2.50 cell).
obj≈80px -> [(8, 10.0), (16, 5.0), (32, 2.5)] | Öneri: stride 32 seviyesi yeterli (≈2.50 cell).



---

## 5) Blok seçimi: “ne ekleyeyim?” yerine “hangi problemi çözmek istiyorum?”

Backbone blokları tipik olarak şu problemleri hedefler:

- **Residual:** gradient akışı, derinlik
- **CSP:** gradient split, verimlilik, daha stabil
- **Grouped conv / ResNeXt:** kapasiteyi grup üzerinden artırma
- **Depthwise-pointwise:** mobil/latency için büyük kazanç
- **Dilated:** receptive field artırma (ama stride ile karıştırma)
- **RepVGG/RepConv:** inference hızına optimize (re-parameterization)

Pratik YOLO için “genelde” güvenli kombinasyon:
- ConvBNAct (SiLU)
- CSP benzeri split
- stage’lerde stride=2 ile downsample



---

## 6) Attention: nerede, ne kadar?

Backbone’da attention kullanımı iki büyük sınıf:

- **Gating attention (SE/ECA/CBAM/CoordAtt)**: CNN için uygun, nispeten hafif
- **QKV/self-attention**: ağır; sınırlı sayıda

Pratik kurallar:
- Erken katmanda (H×W büyük) attention pahalı → genelde kaçın
- Orta/derin stage’lerde daha mantıklı
- Neck, fusion yaptığı için attention için güçlü aday
- Head tarafında en son düşün

En az riskli yol:
1) Neck’te hafif attention (CBAM/CoordAtt)
2) Backbone mid-stage’de SE/ECA



---

## 7) Norm ve regularization: küçük batch / overfit senaryosu

### Norm
- Batch büyükse BN OK
- Batch küçükse GN veya SyncBN düşün

### Regularization
- Derin residual backbone: **DropPath** iyi “default” aday
- Uzamsal ezber: **DropBlock** (daha nazlı ayar ister)
- Hafif ucuz: SpatialDropout

Not: Bu modüller training loss’u artırabilir; hedef val/test mAP artışı.


In [3]:

def choose_norm(batch_total, multi_gpu=False):
    if batch_total >= 16:
        return "BatchNorm"
    if multi_gpu:
        return "SyncBatchNorm"
    return "GroupNorm"

for b in [2,4,8,16,32]:
    print(b, "->", choose_norm(b))


2 -> GroupNorm
4 -> GroupNorm
8 -> GroupNorm
16 -> BatchNorm
32 -> BatchNorm



---

## 8) “Daha iyi backbone”u bulmanın gerçek yöntemi: Controlled Ablation

Backbone geliştirme süreci:

### Adım-1: Baseline sabitle
- aynı split, aynı augmentation, aynı training schedule
- en az 2 seed

### Adım-2: Tek değişken değiştir
Örn:
- sadece stride planı
- sadece blok tipi (CSP vs Res)
- sadece norm (BN vs GN)
- sadece attention (SE ekle/çıkar)

### Adım-3: Ölçüm
- mAP50-95
- mAP_small/medium/large
- FPS/latency
- VRAM
- (opsiyonel) FP/FN breakdown

Aşağıda örnek ablation planı şablonu var.


In [4]:

try:
    import pandas as pd
    plan = [
        {"Exp": "E0", "Değişiklik": "Baseline", "Neden": "Referans", "Ölç": "mAP_s/m/l, FPS, VRAM"},
        {"Exp": "E1", "Değişiklik": "BN→GN (backbone+neck)", "Neden": "small batch stabilite", "Ölç": "mAP + run std"},
        {"Exp": "E2", "Değişiklik": "Backbone mid: SE", "Neden": "channel seçicilik", "Ölç": "mAP, latency"},
        {"Exp": "E3", "Değişiklik": "Neck: CBAM/CoordAtt", "Neden": "fusion iyileştirme", "Ölç": "mAP_small, FP"},
        {"Exp": "E4", "Değişiklik": "DropPath schedule", "Neden": "derin reg", "Ölç": "val mAP, overfit gap"},
    ]
    display(pd.DataFrame(plan))
except Exception:
    pass


,Exp,Değişiklik,Neden,Ölç
0,E0,Baseline,Referans,"mAP_s/m/l, FPS, VRAM"
1,E1,BN→GN (backbone+neck),small batch stabilite,mAP + run std
2,E2,Backbone mid: SE,channel seçicilik,"mAP, latency"
3,E3,Neck: CBAM/CoordAtt,fusion iyileştirme,"mAP_small, FP"
4,E4,DropPath schedule,derin reg,"val mAP, overfit gap"



---

## 9) Tasarım hatalarını erken yakalamak: pre-flight sanity

Tam eğitim yapmadan önce:

- 1 batch overfit (50–200 iter)
- bbox transform doğrulaması (resize/letterbox)
- output shape–loss uyumu
- grad NaN/Inf kontrolü

Bu testler “boşa training”i engeller.
